In [ ]:
#INPLACE=TRUE MODIFIES ORIGINAL DATA
#IN HEAT MAP BIG VALUE =LIGHT COLOR,SMALL VALUE =DARK COLOR USED IN CONTINUOUS DATA eg(TEMPRETURE)


In [3]:
import pandas as pd

df = pd.read_csv("C:/Users/fa22-bscs-0205/Downloads/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [51]:
# 2. Remove outliers using percentile score
# Identify numerical columns
numerical_cols = df.select_dtypes(include=["float64", "int64"]).columns
for col in numerical_cols:
    lower_bound = df[col].quantile(0.01)  # 1st percentile
    upper_bound = df[col].quantile(0.99)  # 99th percentile
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    df.head()

In [41]:
# 3. Convert text columns to numbers
# Label Encoding for ordinal text columns
label_encoder = LabelEncoder()
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = label_encoder.fit_transform(df[col])
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


In [55]:
# 4. Apply scaling
X = df.drop("HeartDisease", axis=1)  # Features
y = df["HeartDisease"]              # Target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [57]:
# 5. Build SVM model (Standalone and Bagging)
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

bagged_svm = BaggingClassifier(base_estimator=SVC(kernel='rbf'), n_estimators=10, random_state=42)
bagged_svm.fit(X_train, y_train)
y_pred_bagged_svm = bagged_svm.predict(X_test)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [59]:
# 6. Build Decision Tree model (Standalone and Bagging)
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)

bagged_tree = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagged_tree.fit(X_train, y_train)
y_pred_bagged_tree = bagged_tree.predict(X_test)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [61]:
# 7. Compare performances
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Bagged SVM Accuracy:", accuracy_score(y_test, y_pred_bagged_svm))
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_tree))
print("Bagged Decision Tree Accuracy:", accuracy_score(y_test, y_pred_bagged_tree))

SVM Accuracy: 0.8778625954198473
Bagged SVM Accuracy: 0.8625954198473282
Decision Tree Accuracy: 0.8091603053435115
Bagged Decision Tree Accuracy: 0.8244274809160306
